Copyright (c) 2025 Mitsuru Ohno  
Use of this source code is governed by a BSD-3-style  
license that can be found in the LICENSE file.  

## モジュールrxn_reader内の関数の使用例  

In [1]:
# development phase
# Add the parent directory (one level up from the notebook's location) to the Python path
import os
import sys

sys.path.append(os.path.join(os.getcwd(), '..'))

In [ ]:
file_path = '../sample_data/sample_data_1b.csv'  # CSVファイルのパスを指定

In [3]:
# 使用例
from src.rxnfit.rxn_reader import (
    get_reactions, 
    to_chempy_style, 
    reactant_consumption, 
    generate_sys_ode, 
    get_unique_species, 
    rate_constants
)

In [4]:
# 取得した反応を表示
reactions = get_reactions(file_path)
print(reactions)

[[['1', 0.054], [['', 'AcOEt'], ['', 'OHa1']], [['', 'AcOa1'], ['', 'EtOH']], []], [['2', 'k2'], [['', 'AcOiPr'], ['', 'OHa1']], [['', 'AcOa1'], ['', 'iPrOH']], []], [['3', 0.031], [['', 'EGOAc2'], ['2', 'OHa1']], [['2', 'AcOa1'], ['', 'EG']], []]]


In [5]:
# 反応の表記をChemPyで用いる辞書形式に変換
list(map(to_chempy_style, reactions))

[[['1', 0.054], {'AcOEt': 1, 'OHa1': 1}, {'AcOa1': 1, 'EtOH': 1}],
 [['2', 'k2'], {'AcOiPr': 1, 'OHa1': 1}, {'AcOa1': 1, 'iPrOH': 1}],
 [['3', 0.031], {'EGOAc2': 1, 'OHa1': 2.0}, {'AcOa1': 2.0, 'EG': 1}]]

In [6]:
# 取得した反応のリストに、速度式、係数を追記
reactant_eq = list(map(reactant_consumption, reactions))
reactant_eq

[[['1', 0.054],
  [['', 'AcOEt'], ['', 'OHa1']],
  [['', 'AcOa1'], ['', 'EtOH']],
  [],
  ['k1*AcOEt(t)*OHa1(t)', ['', ''], ['', '']]],
 [['2', 'k2'],
  [['', 'AcOiPr'], ['', 'OHa1']],
  [['', 'AcOa1'], ['', 'iPrOH']],
  [],
  ['k2*AcOiPr(t)*OHa1(t)', ['', ''], ['', '']]],
 [['3', 0.031],
  [['', 'EGOAc2'], ['2', 'OHa1']],
  [['2', 'AcOa1'], ['', 'EG']],
  [],
  ['k3*EGOAc2(t)*OHa1(t)**2', ['', '2'], ['2', '']]]]

In [7]:
# generate_ode(reaction)
sys_odes_dict = generate_sys_ode(reactant_eq)
sys_odes_dict

{'AcOEt': '-k1*AcOEt(t)*OHa1(t)',
 'OHa1': '-k1*AcOEt(t)*OHa1(t)-k2*AcOiPr(t)*OHa1(t)-2*k3*EGOAc2(t)*OHa1(t)**2',
 'AcOa1': 'k1*AcOEt(t)*OHa1(t)+k2*AcOiPr(t)*OHa1(t)+2*k3*EGOAc2(t)*OHa1(t)**2',
 'EtOH': 'k1*AcOEt(t)*OHa1(t)',
 'AcOiPr': '-k2*AcOiPr(t)*OHa1(t)',
 'iPrOH': 'k2*AcOiPr(t)*OHa1(t)',
 'EGOAc2': '-k3*EGOAc2(t)*OHa1(t)**2',
 'EG': 'k3*EGOAc2(t)*OHa1(t)**2'}

In [8]:
# 反応に関与する化学種を列挙
unique_species_list = get_unique_species(reactant_eq)
unique_species_list

['AcOEt', 'OHa1', 'AcOa1', 'EtOH', 'AcOiPr', 'iPrOH', 'EGOAc2', 'EG']

In [9]:
# 速度定数を辞書形式で列挙
rate_constants_dict = rate_constants(reactant_eq)
rate_constants_dict

{'k1': 0.054, 'k2': 'k2', 'k3': 0.031}